# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [1]:
# Check devices info
import tensorflow as tf

print("[CUDA support]>>>", tf.test.is_built_with_cuda())
print("[GPU support]>>>",tf.test.is_built_with_gpu_support())
print("[ROCm (GPU) support]>>>",tf.test.is_built_with_rocm())

physical_devices = tf.config.list_physical_devices('GPU')
# print("[GPU info]>>>", physical_devices)
if physical_devices:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow.python.client import device_lib
print("[Details]>>>\n",device_lib.list_local_devices())

[CUDA support]>>> True
[GPU support]>>> True
[ROCm (GPU) support]>>> False
[Details]>>>
 [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7671510875167962154
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 18059487753208720946
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15681600800690635560
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7890232935
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13888936683557866659
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
import os, sys, random, math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print(ROOT_DIR)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "keras_model/mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMG_PATH = os.path.join(ROOT_DIR, "test")
# IMG_PATH = os.path.join(ROOT_DIR, "v1/images")
print(IMG_PATH)
if os.path.isdir(IMG_PATH):
    IMAGES_NUM = len(os.listdir(IMG_PATH))
else:
    IMAGES_NUM = 1

/home/tfhub/Projects/Mask_RCNN
/home/tfhub/Projects/Mask_RCNN/test


Using TensorFlow backend.


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [4]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = int(IMAGES_NUM/GPU_COUNT)

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     17
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 17
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE       

## Create Model and Load Trained Weights

In [5]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)
print(model.keras_model, "loaded")

# Save model to folder
# model_ver = 0.01
# tf.saved_model.save(model.keras_model, os.path.join(ROOT_DIR, "/tf_serving/", str(model_ver)))

# model.keras_model.save('stupid_model.h5')

# print(COCO_MODEL_PATH)
# from tensorflow import keras
# new_model = tf.keras.models.load_model(COCO_MODEL_PATH)
# keras.experimental.export_saved_model(model.keras_model, 'stupid.pb')
# tf.keras.models.save_model(model.keras_model, 'stupid.pb')
# tf.saved_model.save(model, 'aha')

# print(model.keras_model)

# freeze and save it to a .pb file
# from keras import backend as K
# K.set_learning_phase(0)
# from tensorflow.python.framework.graph_util import convert_variables_to_constants
# keras_session = K.get_session()
# graph = keras_session.graph
# graph.as_default()
# keep_var_names=None
# output_names=[out.op.name for out in model.keras_model.outputs]
# clear_devices=True
# with graph.as_default():
#     freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
#     output_names = output_names or []
#     output_names += [v.op.name for v in tf.global_variables()]
#     input_graph_def = graph.as_graph_def()
#     if clear_devices:
#         for node in input_graph_def.node:
#             node.device = ""
#     frozen_graph = convert_variables_to_constants(keras_session, input_graph_def,
#                                                   output_names, freeze_var_names)
# tf.train.write_graph(frozen_graph, "model", "my_model.pb", as_text=False)

# print(tf.keras.applications.MobileNet())

<keras.engine.training.Model object at 0x7f7549d68e10> Loaded


## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [5]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

## Run Object Detection

In [6]:
# Load a random image from the images folder
# file_names = next(os.walk(IMG_PATH))[2]
# image = skimage.io.imread(os.path.join(IMG_PATH, random.choice(file_names)))

image = []
if os.path.isdir(IMG_PATH):
    for file in os.listdir(IMG_PATH):
        image.append(skimage.io.imread(os.path.join(IMG_PATH, file)))
else:
    image.append(skimage.io.imread(IMG_PATH))

# Run detection
results = model.detect(image, verbose=1)

# Visualize results
for i in range(len(image)):
    r = results[i]
    visualize.display_instances(image[i], r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])